In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from collections import Counter
import nbformat

In [2]:
%run '/home/ec2-user/kaggle/rudolph/dataPrep.ipynb'

Fetching story: The Boy And The Filberts from /author/aesop/short-story/the-boy-and-the-filberts/...
https://americanliterature.com/author/aesop/short-story/the-boy-and-the-filberts/
Story text extracted successfully:
A Boy was given permission to put his hand into a pitcher to get some filberts. But he took such a great fistful that he could not draw his hand out again. There he stood, unwilling to give up a single filbert and yet unable to get them all out at once. Vexed and disappointed he began to cry.
"My boy," said his mother, "be satisfied with half the nuts you have taken and you will easily get your hand out. Then perhaps you may have some more filberts some other time."
Do not attempt too much at once.



Return to theAesoplibrary , or . . . Read the next short story;The Boy And The Nettle
Or read more short stories for kids in ourChildren's Library
Title: The Boy And The Filberts
Text: A Boy was given permission to put his hand into a pitcher to get some filberts. But he too

In [3]:
print(stories[0][0])
print(stories[0][1])

A Boy was given permission to put his hand into a pitcher to get some filberts. But he took such a great fistful that he could not draw his hand out again. There he stood, unwilling to give up a single filbert and yet unable to get them all out at once. Vexed and disappointed he began to cry.
"My boy," said his mother, "be satisfied with half the nuts you have taken and you will easily get your hand out. Then perhaps you may have some more filberts some other time."
Do not attempt too much at once.



Return to theAesoplibrary , or . . . Read the next short story;The Boy And The Nettle
Or read more short stories for kids in ourChildren's Library
into other to easily out But his theAesoplibrary , Library them great out have There at mother, stories The took you was with hand to may his Boy Nettle your not at boy," and not "My a taken stood, once. said up next filbert once. or began and perhaps fistful Vexed you that too single . more half put he ourChildren's filberts satisfied much . D

In [11]:
# Sample dataset: Pairs of original and scrambled sentences 
"""
sentences = [
    ("I love programming", "programming I love"),
    ("Keras makes deep learning easy", "makes easy Keras deep learning"),
    ("The cat sat on the mat", "on the mat The cat sat"),
    ("Hello world", "world Hello"),
    ("Pine Cone and Pepper Pot ran after her. Jeremy Mouse only stuck his head out of the snow cave. He didn’t want Jack Frost to nip his toes and certainly not his tail! Jack Frost! Jack Frost! cried Tiptoes. We’ve come to pay a visit. Jack Frost landed next to them. He was pale, icy blue, and his arms and fingers were long and glistening. His eyes were sharp, and his wings crackled like breaking ice whenever he moved. He looked so fierce that Pine Cone and Pepper Pot were afraid to open their mouths. It became so chilly they wrapped their beards round their necks and pulled their hats over their ears.", "snow he beards only fingers so out their We’ve of and came his sharp, they Cone her. afraid glistening. he Jack long, their It Jack He open his Pot! was to Pot and hats nip after not He necks the ears. Jeremy over Jack next icy Mouse round moved. He tail! was pale, the pay their icy pine wrapped their didn’t heads ice fierce glistening. mouths. breaking over pay Jack next like arms his moved. and certainly! only out was the look Tiptoes over not mouths. and he his to Pepper afraid cave. him Jeremy stuck crackled came blue, pulled ran his round with tail! He them. Mouse after didn’t nip fingers wings their not tail! mouse Pot afraid stuck arms of toes icy Cone was them pay. Pine wings came Pepper their He fierce long his wings Cone hats icy his over and next certainly Jack after ears. glistening. pay tiptoes! like pine ran was icy cave! certainly moved the."),
    ("Machine learning is fascinating", "fascinating Machine is learning"),
]
"""
sentences = stories

In [ ]:
#print(sentences[0])
#print(stories[0])


('I love programming', 'programming I love')
('A Boy was given permission to put his hand into a pitcher to get some filberts. But he took such a great fistful that he could not draw his hand out again. There he stood, unwilling to give up a single filbert and yet unable to get them all out at once. Vexed and disappointed he began to cry.\n"My boy," said his mother, "be satisfied with half the nuts you have taken and you will easily get your hand out. Then perhaps you may have some more filberts some other time."\nDo not attempt too much at once.\n\n\n\nReturn to theAesoplibrary , or . . . Read the next short story;The Boy And The Nettle\nOr read more short stories for kids in ourChildren\'s Library', 'into other to easily out But his theAesoplibrary , Library them great out have There at mother, stories The took you was with hand to may his Boy Nettle your not at boy," and not "My a taken stood, once. said up next filbert once. or began and perhaps fistful Vexed you that too single . 

In [19]:
# Add <start> and <end> tokens to the original sentences
original_sentences = [f"<start> {s[0]} <end>" for s in sentences]
scrambled_sentences = [s[1] for s in sentences]

# Tokenize and preprocess sentences
tokenizer = Tokenizer(filters="")
tokenizer.fit_on_texts(original_sentences + scrambled_sentences)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

max_seq_length = max(
    max(len(seq.split()) for seq in scrambled_sentences),
    max(len(seq.split()) for seq in original_sentences),
)

# Convert sentences to sequences and pad
input_sequences = pad_sequences(tokenizer.texts_to_sequences(scrambled_sentences), maxlen=max_seq_length, padding="post")
target_sequences = pad_sequences(tokenizer.texts_to_sequences(original_sentences), maxlen=max_seq_length, padding="post")

# Create target sequences for teacher forcing
target_input_sequences = target_sequences[:, :-1]
target_output_sequences = target_sequences[:, 1:]

# Define model parameters
embedding_dim = 200
latent_dim = 200

# Define Encoder
encoder_inputs = Input(shape=(max_seq_length,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(latent_dim, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Define Decoder
decoder_inputs = Input(shape=(max_seq_length - 1,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm, _, _ = LSTM(latent_dim, return_sequences=True, return_state=True)(decoder_embedding, initial_state=encoder_states)
decoder_outputs = Dense(vocab_size, activation="softmax")(decoder_lstm)

# Define model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Prepare data for training
decoder_target_output = np.expand_dims(target_output_sequences, -1)
X_train, X_test, y_train, y_test, decoder_y_train, decoder_y_test = train_test_split(
    input_sequences, target_input_sequences, decoder_target_output, test_size=0.2, random_state=42
)

# Train the model
print("Training the model...")
model.fit(
    [X_train, y_train], decoder_y_train,
    validation_data=([X_test, y_test], decoder_y_test),
    batch_size=50,
    epochs=60,
)

# Define inference models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_outputs, state_h, state_c = LSTM(latent_dim, return_sequences=True, return_state=True)(
    decoder_embedding, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = Dense(vocab_size, activation="softmax")(decoder_lstm_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)





Training the model...
Epoch 1/60


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 8.7604e-04 - loss: 6.7581 - val_accuracy: 0.5756 - val_loss: 6.6518
Epoch 2/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 710ms/step - accuracy: 0.3473 - loss: 6.6940 - val_accuracy: 0.5782 - val_loss: 6.4925
Epoch 3/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 696ms/step - accuracy: 0.3574 - loss: 6.5979 - val_accuracy: 0.5782 - val_loss: 6.1995
Epoch 4/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 703ms/step - accuracy: 0.3522 - loss: 6.4228 - val_accuracy: 0.5769 - val_loss: 5.7085
Epoch 5/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 695ms/step - accuracy: 0.3465 - loss: 6.1307 - val_accuracy: 0.5769 - val_loss: 5.1390
Epoch 6/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 700ms/step - accuracy: 0.3460 - loss: 5.7897 - val_accuracy: 0.5769 - val_loss: 4.6398
Epoch 7/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 703ms/step - accuracy: 0.3460 - loss: 5.4843 - val_accuracy: 0.5769 - val_loss: 4.2081
Epoch 8/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3460 - loss: 5.1930 - val_accuracy: 0.5769 - val_loss: 3.8661
Epoch

In [20]:
# Define function to decode scrambled sentences
def decode_sequence(input_seq, scrambled_sentence):
    """
    Decode a scrambled sentence while ensuring the output is a permutation
    of the words in the scrambled sentence.
    """
    # Encode the input as state vectors
    states_value = encoder_model.predict(input_seq)

    # Tokenize the scrambled sentence to get word counts
    scrambled_words = scrambled_sentence.split()
    word_count = Counter(scrambled_words)

    # Generate the initial target sequence with the start token
    target_seq = np.zeros((1, max_seq_length - 1))
    target_seq[0, 0] = tokenizer.word_index["<start>"]

    # Initialize variables for decoding
    stop_condition = False
    decoded_sentence = []

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Create a mask for valid words
        valid_word_indices = []
        for word, count in word_count.items():
            if count > 0:  # Only include words still available in scrambled input
                token_index = tokenizer.word_index.get(word, None)
                if token_index is not None:
                    valid_word_indices.append(token_index)

        # Mask out probabilities of invalid words
        if valid_word_indices:
            mask = np.ones(output_tokens[0, -1, :].shape, dtype=bool)
            mask[valid_word_indices] = False  # Only allow valid word indices
            output_tokens[0, -1, mask] = 0
        else:
            # No valid words left, terminate decoding
            stop_condition = True
            continue

        # Resample a valid word
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word.get(sampled_token_index, None)

        # Debugging prints
        print(f"Resampled token index: {sampled_token_index}")
        print(f"Resampled word: {sampled_word}")

        # Stop conditions
        if (
            sampled_word == "<end>"
            or len(decoded_sentence) >= max_seq_length - 1
            or sum(word_count.values()) == 0  # Stop when all words are used
        ):
            stop_condition = True
        else:
            # Add the valid word to the output and decrement its count
            if word_count.get(sampled_word, 0) > 0:
                decoded_sentence.append(sampled_word)
                word_count[sampled_word] -= 1

            # Update the target sequence
            target_seq = np.zeros((1, max_seq_length - 1))
            target_seq[0, 0] = sampled_token_index

            # Update states
            states_value = [h, c]

    return " ".join(decoded_sentence).strip()


In [21]:
# Test the model
scrambled_input = "on the mat the cat sat"
input_sequence = pad_sequences(tokenizer.texts_to_sequences([scrambled_input]), maxlen=max_seq_length, padding="post")
unscrambled_sentence = decode_sequence(input_sequence, scrambled_input)
print(f"Scrambled: {scrambled_input}")
print(f"Unscrambled: {unscrambled_sentence}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
Resampled token index: 23
Resampled word: on
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Resampled token index: 1
Resampled word: the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Resampled token index: 1
Resampled word: the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Resampled token index: 91
Resampled word: sat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Scrambled: on the mat the cat sat
Unscrambled: on the the sat


In [ ]:
"""
scrambled_sentence = "on the mat The cat sat"
scrambled_words = scrambled_sentence.split()
print(Counter(scrambled_words))
print(original_sentences)
print(scrambled_sentences)
"""

Counter({'on': 1, 'the': 1, 'mat': 1, 'The': 1, 'cat': 1, 'sat': 1})
['<start> I love programming <end>', '<start> Keras makes deep learning easy <end>', '<start> The cat sat on the mat <end>', '<start> Hello world <end>', '<start> Machine learning is fascinating <end>']
['programming I love', 'makes easy Keras deep learning', 'on the mat The cat sat', 'world Hello', 'fascinating Machine is learning']
